# Генерация эмоциональной музыки

Здесь попробуем генерить музыку с заранее заданной эмоцией, используя предобученную на LMD GPT2 из статьи https://arxiv.org/abs/2008.06048 и идею условной генерации текста из статьи https://arxiv.org/abs/2011.04000

In [1]:
import os

import json

import pandas as pd

import torch
from transformers import GPT2Config, GPT2LMHeadModel

from miditok import REMI

In [2]:
# Зададим некоторые констатные значения

# файл, где хранятся списки тактов по эмоциям
BAG_OF_BARS_FILE = os.path.join('emotion_classification', 'linear_clf', 'bar_weights.xlsx')
# Список возможных эмоций
EMOTIONS = ['cheerful',
            'tense',
            'bizarre']
# эмоция, с которой хотим генерить музыку
EMOTION = EMOTIONS[0]
device = "cuda" if torch.cuda.is_available() and not no_cuda else "cpu"

In [3]:
# инициализируем конфиг для GPT2 используя параметры модели из статьи MMM
gpt2config = GPT2Config(vocab_size=647,
                        n_embd=512,
                        n_head=8,
                        n_positions=2048,
                        n_layer=6
                       )

model = GPT2LMHeadModel(gpt2config)
# Загружаем веса модели
model.load_state_dict(torch.jit.load('models/model.pt').state_dict())
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(647, 512)
    (wpe): Embedding(2048, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropo

In [4]:
# Замораживаем веса
for param in model.parameters():
    param.requires_grad = False

In [5]:
# Загрузим словарь токен - индекс токена для предобученной модели из статьи https://arxiv.org/abs/2008.06048
with open(os.path.join('models', 'tokenizer.json'), 'r') as hfile:
    tokenizer = json.load(hfile)
tokenizer

{'PIECE_START = 0': 0,
 'PIECE_START = 1': 1,
 'NUM_BARS = 4': 2,
 'NUM_BARS = 8': 3,
 'BAR = 0': 4,
 'BAR_END = 0': 5,
 'TIME_SIGNATURE = 1/2': 6,
 'TIME_SIGNATURE = 1/4': 7,
 'TIME_SIGNATURE = 1/8': 8,
 'TIME_SIGNATURE = 1/16': 9,
 'TIME_SIGNATURE = 2/1': 10,
 'TIME_SIGNATURE = 2/2': 11,
 'TIME_SIGNATURE = 2/4': 12,
 'TIME_SIGNATURE = 2/8': 13,
 'TIME_SIGNATURE = 3/1': 14,
 'TIME_SIGNATURE = 3/2': 15,
 'TIME_SIGNATURE = 3/4': 16,
 'TIME_SIGNATURE = 3/8': 17,
 'TIME_SIGNATURE = 4/2': 18,
 'TIME_SIGNATURE = 4/4': 19,
 'TIME_SIGNATURE = 4/8': 20,
 'TIME_SIGNATURE = 5/4': 21,
 'TIME_SIGNATURE = 5/8': 22,
 'TIME_SIGNATURE = 6/2': 23,
 'TIME_SIGNATURE = 6/4': 24,
 'TIME_SIGNATURE = 6/8': 25,
 'TIME_SIGNATURE = 7/4': 26,
 'TIME_SIGNATURE = 7/8': 27,
 'TIME_SIGNATURE = 8/4': 28,
 'TIME_SIGNATURE = 8/8': 29,
 'TIME_SIGNATURE = 9/4': 30,
 'TIME_SIGNATURE = 9/8': 31,
 'TIME_SIGNATURE = 9/16': 32,
 'TIME_SIGNATURE = 10/4': 33,
 'TIME_SIGNATURE = 10/8': 34,
 'TIME_SIGNATURE = 11/8': 35,
 'TIME_SI

In [6]:
# подготовим контекст
# Так как мы генерим без затравок, то в качестве контекста будем использовать токен начала пьесы PIECE_START
context = [tokenizer['PIECE_START = 0']]
# преобразуем его в тензор и добавим размерностей
context_t = torch.tensor(context, device=device, dtype=torch.long)
while len(context_t.shape) < 2:
    context_t = context_t.unsqueeze(0)
# в переменной output_so_far хранятся сгенерированные токены
output_so_far = context_t

output_so_far

tensor([[0]])

В подход, описанном в статье https://arxiv.org/abs/2011.04000 для контроля за генерацией используются списки слов. А у каждого слова есть свой токен. Следовательно, после каждого сгенерированного слова считается лосс и корректируется выход модели.

В нашем случае есть списки тактов по эмоциям. Значит корректировать выход модели нужно будет только после генерации целого такта.

In [13]:
def get_affect_words_and_int(emotion):
    """
    Нужно получить такты, соответствующие эмоциям
    :param emotion: конкретная эмоция, такты для которой ищем
    :return:
    """
    bag_of_bars = pd.read_excel(BAG_OF_BARS_FILE, index_col=0)
    bag_of_bars = bag_of_bars.sort_values(by=emotion)

    bars = bag_of_bars.tail(1000)

    return list(bars.index), list(bars[emotion].values)

In [14]:
affect_bars, affect_int = get_affect_words_and_int(EMOTION)

**Проблема:** токенизация в этих списках тактов отличается от токенизации, используемой в модели. 
**Возможное решение:** Надо найти способ "перевести" токены с языка REMI на язык MMM либо переделать классификацию, используя токенизацию MMM. Последний вариант предпочтительнее, так как проще.

Токенизация в mmm_api происходит на языке C, поэтому проблематично извлечь алгоритм оттуда

**Ещё одно возможное решение:** перевести такты, закодированные REMI в миди и их токенизировать с помощью MMM токенизатора

In [20]:
remi_bars = list(map(lambda elem: elem[-1] + elem[:-2], affect_bars))

In [16]:
tokenizer = REMI()

In [22]:
tok_vocab = tokenizer.vocab
tok_vocab = {key.lower():val for key, val in tok_vocab.items()}

In [34]:
for index, bar in enumerate(remi_bars):
    bar_tokens = [tok_vocab[token] for token in bar.split(' ')]
    midi = tokenizer.tokens_to_midi([bar_tokens])
    midi.dump(os.path.join('data', 'emotion_bars', EMOTION, f'bar_{index}.mid'))
    

Токенизатор не может закодировать midi, полученные из REMI, значит будем переделывать классификацию.

In [30]:
midi = tokenizer.tokens_to_midi([bar_tokens])

In [32]:
midi.dump('test2.mid')